**Projeto de Graduação** 

*Aluno: Victor Martins de Araújo*

*Orientador: Reinaldo Garcia Cipriano*

*Política (t,s,S)*

*Simulação para otimização de uma política de estoque baseada em estratégia mista, ponto de pedido (s), ritmo de pedido (t) e no limite superior de estoque (S)*

In [ ]:
import pandas as pd
import numpy as np
#from google.colab import files

In [ ]:
#Definindo custo de transporte
def c(ftl,quantity):
  ftl_cost = 3100
  if quantity <= ftl:
      return ftl_cost
  elif quantity > ftl:
      num_cars = quantity//ftl
  if quantity % ftl != 0:
      num_cars += 1

  return num_cars * ftl_cost

In [ ]:
#Definindo função de cálculo de custo de estoque
def h(quantity,unit_price):
 stock_cost = quantity * unit_price * 0.5247
 return stock_cost

Estratégia Baseada no ritmo de consumo do estoque em dias e no ponto de pedido (t,s,S)

In [ ]:
#Definindo função de simulação 
def simulation_formula_tsS():
 #definindo os parâmetros (tempo de simulação, lead time,estoque incicial, políticas atuais)
 T = 365 
 unit_price = 1.17
 ftl = 640

 #definindo variáveis da simulação produto 8864
 
 holding_costs = 0
 order_costs = 0
 total_cost = 0
 missing_count = 0
 t1 = 0
 t = 0
 order = 0
 order_count = 0

 #definindo tabela do dataframe do output
 Resultados_tsS = pd.DataFrame(columns = ['Política','Custo Total','Custo Estoque','Custo Pedido','N Pedidos','N Faltas'])

 #início do loop para combinação das diferentes políticas no tempo T
 for s_max in range (5100,100000,500):
    
    for s_min in range(600,25000,500):

        for temp in range (1,30,1):
            hct = []
            oct = []
            tc = []
            tp = []
            tf = []

            for i in range (1):

                total_cost = 0
                order_costs = 0
                stock = 16996 #estoque inicial
                missing_count = 0
                order_count = 0
                order = 0
                holding_costs = 0

                #Simula a dinâmica do estoque para o período de um ano
                for i in range(1,T):

                    holding_costs += h(stock,unit_price)

                    #falta de estoque
                    if stock < 0:
                        missing_count += 1
                        stock = 0

                    #gerando lead time
                    #L = np.random.normal(4, 3) 
                    L = 4
                    #gerando demanda estocástica de produção 
                    demand = np.random.normal(214,109)

                    #projetando estoque
                    stock -= demand

                    #simulando a chegada do material em estoque
                    if i == t1 and order > 0:
                        t1 = 0
                        stock += order
                        order = 0

                    #simulando pedido  baseado no ponto de pedido (s_min)
                    if stock < s_min and order == 0:
                        t1 = i + L
                        order = s_max - stock
                        order_costs += c(ftl,order)
                        order_count +=1

                    #simulando pedido  baseado no rítimo de pedidos (t)
                    if t == temp and order == 0:
                          #gerando lead time estocástico
                          L = np.random.normal(7.76,2.64) 
                          t1 = i + L
                          t = 0
                          order = s_max - stock
                          order_costs += c(ftl,order)
                          order_count +=1

                    #atualiza o contador de tempo  
                    t += 1

                total_cost = holding_costs + order_costs
                hct.append(holding_costs)
                oct.append(order_costs)
                tc.append(total_cost)
                tp.append(order_count)
                tf.append(missing_count)
                    
                #Inserindo dados resultantes no DataFrame 
                Resultados_tsS = Resultados_tsS.append({'Política': [temp,s_max],
                                                'Custo Total': sum(tc)/len(tc),
                                                'Custo Estoque': sum(hct)/len(hct),
                                                'Custo Pedido': sum(oct)/len(oct),
                                                'N Pedidos': sum(tp)/len(tp),
                                                'N Faltas':sum(tf)/len(tf)},
                                                ignore_index=True)

       

 #exportar excel com dados da simulação
 output = Resultados_tsS.to_excel('Resultados_tsS.xlsx')

 #files.download('Resultados_tsS.xlsx')
 return output
 return Resultados_tsS


In [12]:
#chamada da função de simulação
simulation_formula_tsS()

KeyboardInterrupt: 